In [1]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

import warnings
warnings.simplefilter('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Data Mining/pro_hw_1/

/content/drive/MyDrive/Data Mining/pro_hw_1


In [4]:
transaction_df = pd.read_csv("./FoodMart-Transactions-1998.csv")
customer_df = pd.read_csv("./Customer-Lookup.csv")
product_df = pd.read_csv("./Product-Lookup.csv")
display(transaction_df)
display(customer_df)
display(product_df)

,transaction_date,stock_date,product_id,customer_id,store_id,quantity
0,1/1/1998,12/25/1997,4,2439,10,3
1,1/1/1998,12/28/1997,11,4284,10,3
2,1/1/1998,12/28/1997,12,534,10,3
3,1/1/1998,12/29/1997,14,9743,10,2
4,1/1/1998,12/27/1997,16,3608,10,3
...,...,...,...,...,...,...
182878,12/30/1998,12/29/1998,1521,7197,11,3
182879,12/30/1998,12/23/1998,1536,5223,10,2
182880,12/30/1998,12/23/1998,1542,8077,10,4
182881,12/30/1998,12/28/1998,1544,4485,10,2


,customer_id,customer_acct_num,first_name,last_name,customer_address,customer_city,customer_state_province,customer_postal_code,customer_country,birthdate,marital_status,yearly_income,gender,total_children,num_children_at_home,education,acct_open_date,member_card,occupation,homeowner
0,1,87462024688,Sheri,Nowmer,2433 Bailey Road,Tlaxiaco,Oaxaca,15057,Mexico,8/26/1961,M,$30K - $50K,F,4,2,Partial High School,9/10/1991,Bronze,Skilled Manual,Y
1,2,87470586299,Derrick,Whelply,2219 Dewing Avenue,Sooke,BC,17172,Canada,7/3/1915,S,$70K - $90K,M,1,0,Partial High School,3/11/1993,Bronze,Professional,N
2,3,87475757600,Jeanne,Derry,7640 First Ave.,Issaquah,WA,73980,USA,6/21/1910,M,$50K - $70K,F,1,1,Bachelors Degree,6/11/1991,Bronze,Professional,Y
3,4,87500482201,Michael,Spence,337 Tosca Way,Burnaby,BC,74674,Canada,6/20/1969,M,$10K - $30K,M,4,4,Partial High School,5/21/1994,Normal,Skilled Manual,N
4,5,87514054179,Maya,Gutierrez,8668 Via Neruda,Novato,CA,57355,USA,5/10/1951,S,$30K - $50K,F,3,0,Partial College,8/21/1992,Silver,Manual,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10276,10277,87439274191,Fran,Ross,5603 Blackridge Drive,Lake Oswego,OR,52724,USA,2/9/1974,M,$90K - $110K,M,4,3,Partial High School,3/14/1991,Bronze,Management,N
10277,10278,87448420500,Myreda,Calahoo,263 La Orinda Pl.,N. Vancouver,BC,71758,Canada,12/8/1926,M,$30K - $50K,F,0,0,Partial College,3/20/1992,Bronze,Professional,N
10278,10279,87453135848,Mary,Ayers,6885 Auburn,Lincoln Acres,CA,42550,USA,5/18/1913,S,$130K - $150K,M,3,0,Partial High School,11/22/1991,Bronze,Management,Y
10279,10280,87458639740,Ernest,Aiello,5077 Bannock Ct.,Puyallup,WA,27746,USA,9/6/1968,M,$150K +,F,5,2,High School Degree,5/26/1991,Golden,Professional,Y


,product_id,product_brand,product_name,product_sku,product_retail_price,product_cost,product_weight,recyclable,low_fat
0,1,Washington,Washington Berry Juice,90748583674,2.85,0.94,8.39,NaN,NaN
1,2,Washington,Washington Mango Drink,96516502499,0.74,0.26,7.42,NaN,1.0
2,3,Washington,Washington Strawberry Drink,58427771925,0.83,0.40,13.10,1.0,1.0
3,4,Washington,Washington Cream Soda,64412155747,3.64,1.64,10.60,1.0,NaN
4,5,Washington,Washington Diet Soda,85561191439,2.19,0.77,6.66,1.0,NaN
...,...,...,...,...,...,...,...,...,...
1555,1556,CDR,CDR Creamy Peanut Butter,29538288712,2.65,1.14,6.94,1.0,1.0
1556,1557,CDR,CDR Strawberry Preserves,50687324404,1.20,0.48,15.40,1.0,NaN
1557,1558,CDR,CDR Extra Chunky Peanut Butter,84930775761,2.16,0.82,11.50,NaN,1.0
1558,1559,CDR,CDR Apple Preserves,75317577719,1.62,0.62,21.00,NaN,NaN


# Q1
從 Foodmart Data 的交易資料中，探勘符合 Minimum Support = 0.00015 且 Minimum Confidence = 0.8 的 Association Rules，並列出 Confidence 最高的前 10 條 Rules 以及 lift 最高的前 10 條，並比較這兩者的異同。

In [5]:
# Data Clean - merge the transaction and products table
trans = transaction_df[['transaction_date', 'product_id', 'customer_id', 'store_id']]
prod = product_df[['product_id', 'product_name']]

trans_product = pd.merge(trans, prod, on=['product_id']) #Inner Join
display(trans_product)

,transaction_date,product_id,customer_id,store_id,product_name
0,1/1/1998,4,2439,10,Washington Cream Soda
1,1/5/1998,4,9390,7,Washington Cream Soda
2,1/10/1998,4,7227,19,Washington Cream Soda
3,1/10/1998,4,3693,3,Washington Cream Soda
4,1/12/1998,4,10271,8,Washington Cream Soda
...,...,...,...,...,...
182878,12/9/1998,1042,6053,7,PigTail Ice Cream Sandwich
182879,12/9/1998,1042,1787,12,PigTail Ice Cream Sandwich
182880,12/22/1998,1042,1641,10,PigTail Ice Cream Sandwich
182881,12/23/1998,1042,6010,21,PigTail Ice Cream Sandwich


In [6]:
# Drop productID & group the transaction with date, customer, store
trans_product = trans_product[['transaction_date', 'customer_id', 'store_id', 'product_name']]

trans_product = trans_product.groupby(['transaction_date', 'customer_id', 'store_id'])['product_name'].apply(list)
display(trans_product)

transaction_date  customer_id  store_id
1/1/1998          534          10          [Jeffers Oatmeal, Robust Monthly Sports Magazi...
                  1013         10          [Club 1% Milk, Faux Products Whitening Toothpa...
                  1325         10          [Club Havarti Cheese, Club Strawberry Yogurt, ...
                  1425         10          [Club Jack Cheese, Golden Lemon Popsicles, Den...
                  1550         10          [Blue Label Noodle Soup, Bird Call 200 MG Acet...
                                                                 ...                        
9/9/1998          9638         11          [Good Chablis Wine, Consolidated Buffered Aspi...
                  9788         13          [Carrington Fajita French Fries, Tell Tale Gar...
                  9871         13          [Big Time Frozen Mushroom Pizza, Hilltop Laund...
                  10118        13          [Nationeel Avocado Dip, Moms Sliced Ham, Red W...
                  10186       

In [7]:
# Transform products name to one-hot encoding
TE = TransactionEncoder()
trans_product_te = TE.fit(trans_product).transform(trans_product)
trans_product_te_df = pd.DataFrame(trans_product_te, columns=TE.columns_)
display(trans_product_te_df)

,ADJ Rosy Sunglasses,Akron City Map,Akron Eyeglass Screwdriver,American Beef Bologna,American Chicken Hot Dogs,American Cole Slaw,American Corned Beef,American Foot-Long Hot Dogs,American Low Fat Bologna,American Low Fat Cole Slaw,...,Washington Apple Juice,Washington Berry Juice,Washington Cola,Washington Cranberry Juice,Washington Cream Soda,Washington Diet Cola,Washington Diet Soda,Washington Mango Drink,Washington Orange Juice,Washington Strawberry Drink
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37846,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
37847,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
37848,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
37849,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [8]:
# select the threshold - support & cofidence
conditions = {"Min Support": 0.00015, "Min Confidence": 0.8}

apriori_df = apriori(trans_product_te_df, min_support=conditions["Min Support"], use_colnames=True, low_memory=True)
display(apriori_df)

association_df = association_rules(apriori_df , metric="confidence", min_threshold=conditions["Min Confidence"])
display(association_df)

,support,itemsets
0,0.003699,(ADJ Rosy Sunglasses)
1,0.002933,(Akron City Map)
2,0.003435,(Akron Eyeglass Screwdriver)
3,0.002985,(American Beef Bologna)
4,0.002827,(American Chicken Hot Dogs)
...,...,...
3000,0.000185,"(CDR Hot Chocolate, Imagine Low Fat French Fri..."
3001,0.000159,"(Cormorant Scented Toilet Tissue, Hilltop 200 ..."
3002,0.000159,"(High Quality Scissors, Dollar Monthly Sports ..."
3003,0.000159,"(Even Better Sharp Cheddar Cheese, High Top Ne..."


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(BBB Best Tomato Sauce, Imagine Frozen Cheese ...",(Best Choice Apple Fruit Roll),0.000185,0.002933,0.000159,0.857143,292.285714,0.000158,6.979472,0.996763
1,"(Consolidated Mint Mouthwash, Booker Low Fat C...",(Great English Muffins),0.000185,0.004095,0.000159,0.857143,209.314286,0.000158,6.971335,0.995407
2,"(Bravo Fancy Canned Anchovies, Booker Low Fat ...",(High Top Oranges),0.000159,0.003197,0.000159,1.000000,312.818182,0.000158,inf,0.996961
3,"(High Top Oranges, Booker Low Fat String Cheese)",(Bravo Fancy Canned Anchovies),0.000185,0.003382,0.000159,0.857143,253.466518,0.000158,6.976328,0.996239
4,"(CDR Hot Chocolate, Imagine Low Fat French Fries)",(Quick Extra Lean Hamburger),0.000185,0.003117,0.000185,1.000000,320.771186,0.000184,inf,0.997067
5,"(CDR Hot Chocolate, Quick Extra Lean Hamburger)",(Imagine Low Fat French Fries),0.000211,0.002985,0.000185,0.875000,293.094027,0.000184,7.976117,0.996799
6,"(Cormorant Scented Toilet Tissue, Hilltop 200 ...",(Horatio No Salt Popcorn),0.000159,0.003329,0.000159,1.000000,300.404762,0.000158,inf,0.996829
7,"(Cormorant Scented Toilet Tissue, Horatio No S...",(Hilltop 200 MG Acetominifen),0.000185,0.003514,0.000159,0.857143,243.937701,0.000158,6.975404,0.996085
8,"(Hilltop 200 MG Acetominifen, Horatio No Salt ...",(Cormorant Scented Toilet Tissue),0.000185,0.003250,0.000159,0.857143,263.770035,0.000158,6.977253,0.996393
9,"(High Quality Scissors, Dollar Monthly Sports ...",(Plato French Roast Coffee),0.000185,0.003276,0.000159,0.857143,261.642857,0.000158,6.977068,0.996362


In [9]:
# Sort by Confidence
confidence_top10 = association_df.sort_values('confidence', ascending=False).head(10)
display(confidence_top10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
2,"(Bravo Fancy Canned Anchovies, Booker Low Fat ...",(High Top Oranges),0.000159,0.003197,0.000159,1.000000,312.818182,0.000158,inf,0.996961
15,"(Hilltop 200 MG Acetominifen, Just Right Canne...",(Faux Products HCL Nasal Spray),0.000159,0.002959,0.000159,1.000000,337.955357,0.000158,inf,0.997199
4,"(CDR Hot Chocolate, Imagine Low Fat French Fries)",(Quick Extra Lean Hamburger),0.000185,0.003117,0.000185,1.000000,320.771186,0.000184,inf,0.997067
6,"(Cormorant Scented Toilet Tissue, Hilltop 200 ...",(Horatio No Salt Popcorn),0.000159,0.003329,0.000159,1.000000,300.404762,0.000158,inf,0.996829
13,"(Even Better Sharp Cheddar Cheese, High Top Su...",(High Top New Potatos),0.000159,0.003725,0.000159,1.000000,268.446809,0.000158,inf,0.996433
10,"(High Quality Scissors, Plato French Roast Cof...",(Dollar Monthly Sports Magazine),0.000159,0.003197,0.000159,1.000000,312.818182,0.000158,inf,0.996961
5,"(CDR Hot Chocolate, Quick Extra Lean Hamburger)",(Imagine Low Fat French Fries),0.000211,0.002985,0.000185,0.875000,293.094027,0.000184,7.976117,0.996799
0,"(BBB Best Tomato Sauce, Imagine Frozen Cheese ...",(Best Choice Apple Fruit Roll),0.000185,0.002933,0.000159,0.857143,292.285714,0.000158,6.979472,0.996763
11,"(Dollar Monthly Sports Magazine, Plato French ...",(High Quality Scissors),0.000185,0.003223,0.000159,0.857143,265.932084,0.000158,6.977438,0.996424
14,"(High Top New Potatos, High Top Summer Squash)",(Even Better Sharp Cheddar Cheese),0.000185,0.003223,0.000159,0.857143,265.932084,0.000158,6.977438,0.996424


In [10]:
for i in range(len(confidence_top10)):
    index = confidence_top10.index[i]
    row = association_df.iloc[index].values

    print('Top',i+1 ,':')
    print('Antecedents: \n', list(row[0]))
    print('Consequents: \n', list(row[1]))
    print(f'Support: {round(row[4], 5)}, Confidence: {round(row[5], 5)}, Lift: {round(row[6], 5)}')
    print('\n')

Top 1 :
Antecedents: 
 ['Bravo Fancy Canned Anchovies', 'Booker Low Fat String Cheese']
Consequents: 
 ['High Top Oranges']
Support: 0.00016, Confidence: 1.0, Lift: 312.81818


Top 2 :
Antecedents: 
 ['Hilltop 200 MG Acetominifen', 'Just Right Canned Yams']
Consequents: 
 ['Faux Products HCL Nasal Spray']
Support: 0.00016, Confidence: 1.0, Lift: 337.95536


Top 3 :
Antecedents: 
 ['CDR Hot Chocolate', 'Imagine Low Fat French Fries']
Consequents: 
 ['Quick Extra Lean Hamburger']
Support: 0.00018, Confidence: 1.0, Lift: 320.77119


Top 4 :
Antecedents: 
 ['Cormorant Scented Toilet Tissue', 'Hilltop 200 MG Acetominifen']
Consequents: 
 ['Horatio No Salt Popcorn']
Support: 0.00016, Confidence: 1.0, Lift: 300.40476


Top 5 :
Antecedents: 
 ['Even Better Sharp Cheddar Cheese', 'High Top Summer Squash']
Consequents: 
 ['High Top New Potatos']
Support: 0.00016, Confidence: 1.0, Lift: 268.44681


Top 6 :
Antecedents: 
 ['High Quality Scissors', 'Plato French Roast Coffee']
Consequents: 
 ['Doll

In [11]:
# Sort by Lift
lift_top10 = association_df.sort_values('lift', ascending=False).head(10)
display(lift_top10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
15,"(Hilltop 200 MG Acetominifen, Just Right Canne...",(Faux Products HCL Nasal Spray),0.000159,0.002959,0.000159,1.000000,337.955357,0.000158,inf,0.997199
4,"(CDR Hot Chocolate, Imagine Low Fat French Fries)",(Quick Extra Lean Hamburger),0.000185,0.003117,0.000185,1.000000,320.771186,0.000184,inf,0.997067
2,"(Bravo Fancy Canned Anchovies, Booker Low Fat ...",(High Top Oranges),0.000159,0.003197,0.000159,1.000000,312.818182,0.000158,inf,0.996961
10,"(High Quality Scissors, Plato French Roast Cof...",(Dollar Monthly Sports Magazine),0.000159,0.003197,0.000159,1.000000,312.818182,0.000158,inf,0.996961
6,"(Cormorant Scented Toilet Tissue, Hilltop 200 ...",(Horatio No Salt Popcorn),0.000159,0.003329,0.000159,1.000000,300.404762,0.000158,inf,0.996829
5,"(CDR Hot Chocolate, Quick Extra Lean Hamburger)",(Imagine Low Fat French Fries),0.000211,0.002985,0.000185,0.875000,293.094027,0.000184,7.976117,0.996799
0,"(BBB Best Tomato Sauce, Imagine Frozen Cheese ...",(Best Choice Apple Fruit Roll),0.000185,0.002933,0.000159,0.857143,292.285714,0.000158,6.979472,0.996763
12,"(Even Better Sharp Cheddar Cheese, High Top Ne...",(High Top Summer Squash),0.000185,0.003065,0.000159,0.857143,279.687192,0.000158,6.978547,0.996609
13,"(Even Better Sharp Cheddar Cheese, High Top Su...",(High Top New Potatos),0.000159,0.003725,0.000159,1.000000,268.446809,0.000158,inf,0.996433
11,"(Dollar Monthly Sports Magazine, Plato French ...",(High Quality Scissors),0.000185,0.003223,0.000159,0.857143,265.932084,0.000158,6.977438,0.996424


In [12]:
for i in range(len(lift_top10)):
    index = lift_top10.index[i]
    row = association_df.iloc[index].values

    print('Top',i+1 ,':')
    print('Antecedents: \n', list(row[0]))
    print('Consequents: \n', list(row[1]))
    print(f'Support: {round(row[4], 5)}, Confidence: {round(row[5], 5)}, Lift: {round(row[6], 5)}')
    print('\n')

Top 1 :
Antecedents: 
 ['Hilltop 200 MG Acetominifen', 'Just Right Canned Yams']
Consequents: 
 ['Faux Products HCL Nasal Spray']
Support: 0.00016, Confidence: 1.0, Lift: 337.95536


Top 2 :
Antecedents: 
 ['CDR Hot Chocolate', 'Imagine Low Fat French Fries']
Consequents: 
 ['Quick Extra Lean Hamburger']
Support: 0.00018, Confidence: 1.0, Lift: 320.77119


Top 3 :
Antecedents: 
 ['Bravo Fancy Canned Anchovies', 'Booker Low Fat String Cheese']
Consequents: 
 ['High Top Oranges']
Support: 0.00016, Confidence: 1.0, Lift: 312.81818


Top 4 :
Antecedents: 
 ['High Quality Scissors', 'Plato French Roast Coffee']
Consequents: 
 ['Dollar Monthly Sports Magazine']
Support: 0.00016, Confidence: 1.0, Lift: 312.81818


Top 5 :
Antecedents: 
 ['Cormorant Scented Toilet Tissue', 'Hilltop 200 MG Acetominifen']
Consequents: 
 ['Horatio No Salt Popcorn']
Support: 0.00016, Confidence: 1.0, Lift: 300.40476


Top 6 :
Antecedents: 
 ['CDR Hot Chocolate', 'Quick Extra Lean Hamburger']
Consequents: 
 ['Imagi

In [13]:
# Sort by Confidence & Lift
con_lift_top10 = association_df.sort_values(['confidence', 'lift'], ascending=False).head(10)
display(con_lift_top10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
15,"(Hilltop 200 MG Acetominifen, Just Right Canne...",(Faux Products HCL Nasal Spray),0.000159,0.002959,0.000159,1.000000,337.955357,0.000158,inf,0.997199
4,"(CDR Hot Chocolate, Imagine Low Fat French Fries)",(Quick Extra Lean Hamburger),0.000185,0.003117,0.000185,1.000000,320.771186,0.000184,inf,0.997067
2,"(Bravo Fancy Canned Anchovies, Booker Low Fat ...",(High Top Oranges),0.000159,0.003197,0.000159,1.000000,312.818182,0.000158,inf,0.996961
10,"(High Quality Scissors, Plato French Roast Cof...",(Dollar Monthly Sports Magazine),0.000159,0.003197,0.000159,1.000000,312.818182,0.000158,inf,0.996961
6,"(Cormorant Scented Toilet Tissue, Hilltop 200 ...",(Horatio No Salt Popcorn),0.000159,0.003329,0.000159,1.000000,300.404762,0.000158,inf,0.996829
13,"(Even Better Sharp Cheddar Cheese, High Top Su...",(High Top New Potatos),0.000159,0.003725,0.000159,1.000000,268.446809,0.000158,inf,0.996433
5,"(CDR Hot Chocolate, Quick Extra Lean Hamburger)",(Imagine Low Fat French Fries),0.000211,0.002985,0.000185,0.875000,293.094027,0.000184,7.976117,0.996799
0,"(BBB Best Tomato Sauce, Imagine Frozen Cheese ...",(Best Choice Apple Fruit Roll),0.000185,0.002933,0.000159,0.857143,292.285714,0.000158,6.979472,0.996763
12,"(Even Better Sharp Cheddar Cheese, High Top Ne...",(High Top Summer Squash),0.000185,0.003065,0.000159,0.857143,279.687192,0.000158,6.978547,0.996609
11,"(Dollar Monthly Sports Magazine, Plato French ...",(High Quality Scissors),0.000185,0.003223,0.000159,0.857143,265.932084,0.000158,6.977438,0.996424


In [14]:
for i in range(len(con_lift_top10)):
    index = con_lift_top10.index[i]
    row = association_df.iloc[index].values

    print('Top',i+1 ,':')
    print('Antecedents: \n', list(row[0]))
    print('Consequents: \n', list(row[1]))
    print(f'Support: {round(row[4], 5)}, Confidence: {round(row[5], 5)}, Lift: {round(row[6], 5)}')
    print('\n')

Top 1 :
Antecedents: 
 ['Hilltop 200 MG Acetominifen', 'Just Right Canned Yams']
Consequents: 
 ['Faux Products HCL Nasal Spray']
Support: 0.00016, Confidence: 1.0, Lift: 337.95536


Top 2 :
Antecedents: 
 ['CDR Hot Chocolate', 'Imagine Low Fat French Fries']
Consequents: 
 ['Quick Extra Lean Hamburger']
Support: 0.00018, Confidence: 1.0, Lift: 320.77119


Top 3 :
Antecedents: 
 ['Bravo Fancy Canned Anchovies', 'Booker Low Fat String Cheese']
Consequents: 
 ['High Top Oranges']
Support: 0.00016, Confidence: 1.0, Lift: 312.81818


Top 4 :
Antecedents: 
 ['High Quality Scissors', 'Plato French Roast Coffee']
Consequents: 
 ['Dollar Monthly Sports Magazine']
Support: 0.00016, Confidence: 1.0, Lift: 312.81818


Top 5 :
Antecedents: 
 ['Cormorant Scented Toilet Tissue', 'Hilltop 200 MG Acetominifen']
Consequents: 
 ['Horatio No Salt Popcorn']
Support: 0.00016, Confidence: 1.0, Lift: 300.40476


Top 6 :
Antecedents: 
 ['Even Better Sharp Cheddar Cheese', 'High Top Summer Squash']
Consequents

# Q2
有時候我們有興趣的資料不只有產品間的資訊，也會想要由 User Profile 探勘顧客的基本資料。在給定 Minimum Support = 0.05 且 Minimum Confidence = 0.9 的條件下，探勘 Foodmart 顧客基本資料的屬性 {customer_state_province, yearly_income, gender, total_children, num_children_at_home, education, occupation, homeowner} 間的 Association Rules，並列出10條。

In [76]:
cus_df = customer_df[['customer_state_province', 'yearly_income', 'gender', 'total_children', 'num_children_at_home', 'education', 'occupation', 'homeowner']]
display(cus_df)

,customer_state_province,yearly_income,gender,total_children,num_children_at_home,education,occupation,homeowner
0,Oaxaca,$30K - $50K,F,4,2,Partial High School,Skilled Manual,Y
1,BC,$70K - $90K,M,1,0,Partial High School,Professional,N
2,WA,$50K - $70K,F,1,1,Bachelors Degree,Professional,Y
3,BC,$10K - $30K,M,4,4,Partial High School,Skilled Manual,N
4,CA,$30K - $50K,F,3,0,Partial College,Manual,N
...,...,...,...,...,...,...,...,...
10276,OR,$90K - $110K,M,4,3,Partial High School,Management,N
10277,BC,$30K - $50K,F,0,0,Partial College,Professional,N
10278,CA,$130K - $150K,M,3,0,Partial High School,Management,Y
10279,WA,$150K +,F,5,2,High School Degree,Professional,Y


In [77]:
cus_df = cus_df.astype('object')

trans_cus_df = pd.get_dummies(cus_df).astype(bool)
display(trans_cus_df)

,customer_state_province_BC,customer_state_province_CA,customer_state_province_DF,customer_state_province_Guerrero,customer_state_province_Jalisco,customer_state_province_Mexico,customer_state_province_OR,customer_state_province_Oaxaca,customer_state_province_Sinaloa,customer_state_province_Veracruz,...,education_High School Degree,education_Partial College,education_Partial High School,occupation_Clerical,occupation_Management,occupation_Manual,occupation_Professional,occupation_Skilled Manual,homeowner_N,homeowner_Y
0,False,False,False,False,False,False,False,True,False,False,...,False,False,True,False,False,False,False,True,False,True
1,True,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,True,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,True
3,True,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,True,True,False
4,False,True,False,False,False,False,False,False,False,False,...,False,True,False,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10276,False,False,False,False,False,False,True,False,False,False,...,False,False,True,False,True,False,False,False,True,False
10277,True,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,True,False,True,False
10278,False,True,False,False,False,False,False,False,False,False,...,False,False,True,False,True,False,False,False,False,True
10279,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,True,False,False,True


In [78]:
trans_cus_df.columns

Index(['customer_state_province_BC', 'customer_state_province_CA',
       'customer_state_province_DF', 'customer_state_province_Guerrero',
       'customer_state_province_Jalisco', 'customer_state_province_Mexico',
       'customer_state_province_OR', 'customer_state_province_Oaxaca',
       'customer_state_province_Sinaloa', 'customer_state_province_Veracruz',
       'customer_state_province_WA', 'customer_state_province_Yucatan',
       'customer_state_province_Zacatecas', 'yearly_income_$10K - $30K',
       'yearly_income_$110K - $130K', 'yearly_income_$130K - $150K',
       'yearly_income_$150K +', 'yearly_income_$30K - $50K',
       'yearly_income_$50K - $70K', 'yearly_income_$70K - $90K',
       'yearly_income_$90K - $110K', 'gender_F', 'gender_M',
       'total_children_0', 'total_children_1', 'total_children_2',
       'total_children_3', 'total_children_4', 'total_children_5',
       'num_children_at_home_0', 'num_children_at_home_1',
       'num_children_at_home_2', 'num_chi

In [79]:
# select the threshold - support & cofidence
conditions = {"Min Support": 0.05, "Min Confidence": 0.9}

apriori_df = apriori(trans_cus_df, min_support=conditions["Min Support"], use_colnames=True, low_memory=True)
display(apriori_df)

association_df = association_rules(apriori_df , metric="confidence", min_threshold=conditions["Min Confidence"])
display(association_df)

,support,itemsets
0,0.167007,(customer_state_province_BC)
1,0.410660,(customer_state_province_CA)
2,0.102227,(customer_state_province_OR)
3,0.202899,(customer_state_province_WA)
4,0.216127,(yearly_income_$10K - $30K)
...,...,...
476,0.061959,"(gender_M, num_children_at_home_0, occupation_..."
477,0.058068,"(gender_M, num_children_at_home_0, occupation_..."
478,0.053108,"(gender_M, education_Bachelors Degree, occupat..."
479,0.061375,"(num_children_at_home_0, homeowner_N, occupati..."


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(yearly_income_$10K - $30K),(education_Partial High School),0.216127,0.300943,0.200564,0.927993,3.083611,0.135522,9.708147,0.862008
1,(total_children_0),(num_children_at_home_0),0.100963,0.628052,0.100963,1.000000,1.592225,0.037553,inf,0.413718
2,"(yearly_income_$10K - $30K, customer_state_pro...",(education_Partial High School),0.089875,0.300943,0.083455,0.928571,3.085534,0.056408,9.786791,0.742653
3,"(gender_F, yearly_income_$10K - $30K)",(education_Partial High School),0.107869,0.300943,0.099698,0.924256,3.071195,0.067236,9.229210,0.755936
4,"(yearly_income_$10K - $30K, gender_M)",(education_Partial High School),0.108258,0.300943,0.100866,0.931716,3.095984,0.068286,10.237499,0.759189
5,"(num_children_at_home_0, yearly_income_$10K - ...",(education_Partial High School),0.134423,0.300943,0.125182,0.931259,3.094465,0.084729,10.169433,0.781955
6,"(yearly_income_$10K - $30K, occupation_Manual)",(education_Partial High School),0.105145,0.300943,0.101449,0.964847,3.206075,0.069806,19.886318,0.768943
7,"(yearly_income_$10K - $30K, occupation_Skilled...",(education_Partial High School),0.103297,0.300943,0.099115,0.959510,3.188341,0.068028,17.265070,0.765423
8,"(yearly_income_$10K - $30K, homeowner_N)",(education_Partial High School),0.092501,0.300943,0.084914,0.917981,3.050344,0.057076,8.523112,0.740682
9,"(yearly_income_$10K - $30K, homeowner_Y)",(education_Partial High School),0.123626,0.300943,0.115650,0.935484,3.108503,0.078446,10.835376,0.773987


In [80]:
# Sort by Confidence & support
con_sup_top10 = association_df.sort_values(['confidence', 'support'], ascending=False).head(10)
display(con_sup_top10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1,(total_children_0),(num_children_at_home_0),0.100963,0.628052,0.100963,1.000000,1.592225,0.037553,inf,0.413718
13,"(total_children_0, homeowner_Y)",(num_children_at_home_0),0.061278,0.628052,0.061278,1.000000,1.592225,0.022792,inf,0.396228
12,"(total_children_0, gender_M)",(num_children_at_home_0),0.054178,0.628052,0.054178,1.000000,1.592225,0.020151,inf,0.393254
21,"(num_children_at_home_0, yearly_income_$10K - ...",(education_Partial High School),0.067017,0.300943,0.064974,0.969521,3.221605,0.044806,22.935711,0.739130
25,"(yearly_income_$10K - $30K, occupation_Manual,...",(education_Partial High School),0.061959,0.300943,0.060014,0.968603,3.218554,0.041367,22.264950,0.734831
16,"(gender_F, yearly_income_$10K - $30K, occupati...",(education_Partial High School),0.051746,0.300943,0.050092,0.968045,3.216701,0.034520,21.876357,0.726728
6,"(yearly_income_$10K - $30K, occupation_Manual)",(education_Partial High School),0.105145,0.300943,0.101449,0.964847,3.206075,0.069806,19.886318,0.768943
26,"(yearly_income_$10K - $30K, occupation_Skilled...",(education_Partial High School),0.057679,0.300943,0.055637,0.964587,3.205209,0.038278,19.740024,0.730121
19,"(yearly_income_$10K - $30K, occupation_Manual,...",(education_Partial High School),0.053399,0.300943,0.051357,0.961749,3.195778,0.035287,18.275335,0.725847
22,"(num_children_at_home_0, yearly_income_$10K - ...",(education_Partial High School),0.062737,0.300943,0.060208,0.959690,3.188937,0.041328,17.341979,0.732362


# Q3
請探勘 Foodmart Data 中，顧客背景資料與其交易資料之間的關係 (Quantitative Association Rules)。例如 80% 女性顧客常買保養品。請自行設定 Minimum Support、Minimum Confidence， 找出 10 條你覺得有意義的 Rules。請說明你的作法及相關參數設定。

In [20]:
# 顧客ID、商品ID
trans_partial_df = transaction_df[['customer_id', 'product_id']]

# 產品ID、產品名n
product_partial_df = product_df[['product_id', 'product_name']]

# 顧客ID、居住省份、生日、年收入、性別、有幾個小孩、職業
customer_partial_df = customer_df[['customer_id', 'customer_state_province', 'yearly_income', 'gender', 'total_children', 'birthdate', 'occupation']]

In [21]:
display(product_partial_df)
display(customer_partial_df)

,product_id,product_name
0,1,Washington Berry Juice
1,2,Washington Mango Drink
2,3,Washington Strawberry Drink
3,4,Washington Cream Soda
4,5,Washington Diet Soda
...,...,...
1555,1556,CDR Creamy Peanut Butter
1556,1557,CDR Strawberry Preserves
1557,1558,CDR Extra Chunky Peanut Butter
1558,1559,CDR Apple Preserves


,customer_id,customer_state_province,yearly_income,gender,total_children,birthdate,occupation
0,1,Oaxaca,$30K - $50K,F,4,8/26/1961,Skilled Manual
1,2,BC,$70K - $90K,M,1,7/3/1915,Professional
2,3,WA,$50K - $70K,F,1,6/21/1910,Professional
3,4,BC,$10K - $30K,M,4,6/20/1969,Skilled Manual
4,5,CA,$30K - $50K,F,3,5/10/1951,Manual
...,...,...,...,...,...,...,...
10276,10277,OR,$90K - $110K,M,4,2/9/1974,Management
10277,10278,BC,$30K - $50K,F,0,12/8/1926,Professional
10278,10279,CA,$130K - $150K,M,3,5/18/1913,Management
10279,10280,WA,$150K +,F,5,9/6/1968,Professional


In [22]:
# Data Clean - merge the transaction and products table
trans_pro_df = pd.merge(trans_partial_df, product_partial_df, on=['product_id']).drop(['product_id'], axis=1)
customer_id = trans_pro_df[['customer_id']]

# product one-hot
product_onehot = pd.get_dummies(trans_pro_df)
product_onehot_df = product_onehot.groupby(['customer_id']).any().astype(bool)
product_onehot_df = product_onehot_df.reset_index()

display(product_onehot_df)

,customer_id,product_name_ADJ Rosy Sunglasses,product_name_Akron City Map,product_name_Akron Eyeglass Screwdriver,product_name_American Beef Bologna,product_name_American Chicken Hot Dogs,product_name_American Cole Slaw,product_name_American Corned Beef,product_name_American Foot-Long Hot Dogs,product_name_American Low Fat Bologna,...,product_name_Washington Apple Juice,product_name_Washington Berry Juice,product_name_Washington Cola,product_name_Washington Cranberry Juice,product_name_Washington Cream Soda,product_name_Washington Diet Cola,product_name_Washington Diet Soda,product_name_Washington Mango Drink,product_name_Washington Orange Juice,product_name_Washington Strawberry Drink
0,3,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,6,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,8,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,9,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4,10,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8055,10275,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8056,10277,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8057,10278,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8058,10280,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


處理birthdate 將年份獨自提出

In [23]:
customer_partial_df['birth_year'] = customer_partial_df['birthdate'].str.rsplit('/',1).str[-1]
display(customer_partial_df)

,customer_id,customer_state_province,yearly_income,gender,total_children,birthdate,occupation,birth_year
0,1,Oaxaca,$30K - $50K,F,4,8/26/1961,Skilled Manual,1961
1,2,BC,$70K - $90K,M,1,7/3/1915,Professional,1915
2,3,WA,$50K - $70K,F,1,6/21/1910,Professional,1910
3,4,BC,$10K - $30K,M,4,6/20/1969,Skilled Manual,1969
4,5,CA,$30K - $50K,F,3,5/10/1951,Manual,1951
...,...,...,...,...,...,...,...,...
10276,10277,OR,$90K - $110K,M,4,2/9/1974,Management,1974
10277,10278,BC,$30K - $50K,F,0,12/8/1926,Professional,1926
10278,10279,CA,$130K - $150K,M,3,5/18/1913,Management,1913
10279,10280,WA,$150K +,F,5,9/6/1968,Professional,1968


In [24]:
year = customer_partial_df['birth_year']
print(f'Max: {np.max(year)}, Min: {np.min(year)}')

Max: 1980, Min: 1910


In [25]:
for i in range(len(customer_partial_df)):
  year = customer_partial_df['birth_year'][i]
  customer_partial_df['birth_year'][i] = year = customer_partial_df['birth_year'][i][0:3]
  customer_partial_df['birth_year'][i] = '{:<04s}'.format(customer_partial_df['birth_year'][i])

customer_partial_df = customer_partial_df.drop(['birthdate'], axis=1)
display(customer_partial_df)

,customer_id,customer_state_province,yearly_income,gender,total_children,occupation,birth_year
0,1,Oaxaca,$30K - $50K,F,4,Skilled Manual,1960
1,2,BC,$70K - $90K,M,1,Professional,1910
2,3,WA,$50K - $70K,F,1,Professional,1910
3,4,BC,$10K - $30K,M,4,Skilled Manual,1960
4,5,CA,$30K - $50K,F,3,Manual,1950
...,...,...,...,...,...,...,...
10276,10277,OR,$90K - $110K,M,4,Management,1970
10277,10278,BC,$30K - $50K,F,0,Professional,1920
10278,10279,CA,$130K - $150K,M,3,Management,1910
10279,10280,WA,$150K +,F,5,Professional,1960


In [26]:
# 將所有欄位設成object型態
customer_partial_df = customer_partial_df.set_index('customer_id')
customer_partial_df = customer_partial_df.astype(object)

# customer ont-hot
customer_onehot_df = pd.get_dummies(customer_partial_df).astype(bool)
customer_onehot_df = customer_onehot_df.reset_index()

# merge products and customers
all_df = pd.merge(product_onehot_df, customer_onehot_df, on=['customer_id']).drop(['customer_id'], axis=1)
display(all_df)

,product_name_ADJ Rosy Sunglasses,product_name_Akron City Map,product_name_Akron Eyeglass Screwdriver,product_name_American Beef Bologna,product_name_American Chicken Hot Dogs,product_name_American Cole Slaw,product_name_American Corned Beef,product_name_American Foot-Long Hot Dogs,product_name_American Low Fat Bologna,product_name_American Low Fat Cole Slaw,...,occupation_Professional,occupation_Skilled Manual,birth_year_1910,birth_year_1920,birth_year_1930,birth_year_1940,birth_year_1950,birth_year_1960,birth_year_1970,birth_year_1980
0,False,False,False,False,False,False,False,False,False,False,...,True,False,True,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8055,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
8056,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
8057,False,False,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False
8058,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,True,False,False


In [27]:
# select the threshold - support & cofidence
conditions = {"Min Support": 0.001, "Min Confidence": 0.9}

apriori_df = apriori(all_df, min_support=conditions["Min Support"], use_colnames=True, low_memory=True)
display(apriori_df)

association_df = association_rules(apriori_df , metric="confidence", min_threshold=conditions["Min Confidence"])
display(association_df)

,support,itemsets
0,0.016998,(product_name_ADJ Rosy Sunglasses)
1,0.013524,(product_name_Akron City Map)
2,0.016005,(product_name_Akron Eyeglass Screwdriver)
3,0.013648,(product_name_American Beef Bologna)
4,0.013151,(product_name_American Chicken Hot Dogs)
...,...,...
172882,0.001365,"(gender_M, total_children_1, customer_state_pr..."
172883,0.001241,"(birth_year_1940, gender_M, total_children_1, ..."
172884,0.001117,"(birth_year_1960, gender_M, total_children_1, ..."
172885,0.001365,"(total_children_4, birth_year_1930, gender_M, ..."


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(product_name_ADJ Rosy Sunglasses, product_nam...",(gender_F),0.001241,0.493300,0.001117,0.900000,1.824447,0.000505,5.066998,0.452450
1,"(product_name_CDR Brown Sugar, product_name_Ak...",(gender_F),0.001365,0.493300,0.001241,0.909091,1.842875,0.000567,5.573697,0.457995
2,"(product_name_High Top Tomatos, product_name_A...",(gender_F),0.001241,0.493300,0.001117,0.900000,1.824447,0.000505,5.066998,0.452450
3,"(product_name_Dollar Monthly Sports Magazine, ...",(gender_M),0.001241,0.506700,0.001117,0.900000,1.776200,0.000488,4.933002,0.437543
4,"(product_name_Discover Rice Medly, product_nam...",(gender_F),0.001241,0.493300,0.001117,0.900000,1.824447,0.000505,5.066998,0.452450
...,...,...,...,...,...,...,...,...,...,...
1060,"(birth_year_1970, yearly_income_$50K - $70K, t...",(occupation_Professional),0.001241,0.327171,0.001117,0.900000,2.750853,0.000711,6.728288,0.637267
1061,"(total_children_1, yearly_income_$70K - $90K, ...",(occupation_Professional),0.001489,0.327171,0.001365,0.916667,2.801795,0.000878,8.073945,0.644045
1062,"(total_children_4, birth_year_1960, yearly_inc...",(occupation_Professional),0.001613,0.327171,0.001489,0.923077,2.821388,0.000961,8.746774,0.646607
1063,"(total_children_4, birth_year_1970, yearly_inc...",(occupation_Professional),0.001365,0.327171,0.001241,0.909091,2.778640,0.000794,7.401117,0.640986


In [28]:
result = association_df[['antecedents', 'consequents', 'support', 'confidence']]

indexs = []
for index, data in enumerate(result['antecedents']):
  for item in data:
    if "product_name" in item:
      indexs.append(index)

for index, data in enumerate(result['consequents']):
  for item in data:
    if "product_name" in item:
      indexs.append(index)

indexs = list(set(indexs))
result = result.iloc[indexs]

display(result)

,antecedents,consequents,support,confidence
0,"(product_name_ADJ Rosy Sunglasses, product_nam...",(gender_F),0.001117,0.900000
1,"(product_name_CDR Brown Sugar, product_name_Ak...",(gender_F),0.001241,0.909091
2,"(product_name_High Top Tomatos, product_name_A...",(gender_F),0.001117,0.900000
3,"(product_name_Dollar Monthly Sports Magazine, ...",(gender_M),0.001117,0.900000
4,"(product_name_Discover Rice Medly, product_nam...",(gender_F),0.001117,0.900000
...,...,...,...,...
1007,"(total_children_4, product_name_Walrus Merlot ...",(gender_F),0.001241,0.909091
1035,"(total_children_4, gender_F, yearly_income_$70...",(occupation_Professional),0.001117,0.900000
1036,"(total_children_4, yearly_income_$70K - $90K, ...",(gender_F),0.001117,0.900000
1037,"(product_name_Nationeel Chocolate Donuts, gend...",(total_children_4),0.001117,1.000000


In [29]:
# Sort by Confidence & Support
result_top10 = result.sort_values(['confidence', 'support'], ascending=False).head(10)
result_top10 = result_top10.reset_index(drop=True)
display(result_top10)

,antecedents,consequents,support,confidence
0,"(product_name_Big Time Turkey TV Dinner, yearl...",(gender_F),0.001985,1.0
1,"(product_name_Hermanos Elephant Garlic, yearly...",(gender_M),0.001861,1.0
2,(product_name_Even Better Large Curd Cottage C...,(gender_F),0.001613,1.0
3,"(product_name_Plato Strawberry Jam, yearly_inc...",(occupation_Professional),0.001613,1.0
4,"(product_name_Choice Tasty Candy Bar, yearly_i...",(occupation_Professional),0.001613,1.0
5,"(birth_year_1950, product_name_Fast Low Fat Po...",(gender_F),0.001489,1.0
6,"(customer_state_province_OR, product_name_Fram...",(gender_M),0.001489,1.0
7,"(occupation_Management, product_name_Gorilla L...",(gender_M),0.001489,1.0
8,"(product_name_Horatio Chocolate Chip Cookies, ...",(occupation_Professional),0.001489,1.0
9,"(product_name_Red Spade Corned Beef, customer_...",(gender_M),0.001489,1.0


In [30]:
for i in range(len(result_top10)):
    antecedents = result_top10['antecedents'][i]
    consequents = result_top10['consequents'][i]
    support = result_top10['support'][i]
    confidence = result_top10['confidence'][i]

    print('Top',i+1 ,':')
    print('Antecedents: \n', list(antecedents))
    print('Consequents: \n', list(consequents))
    print(f'Support: {round(support, 5)}, Confidence: {round(confidence, 5)}')
    print('\n')

Top 1 :
Antecedents: 
 ['product_name_Big Time Turkey TV Dinner', 'yearly_income_$70K - $90K']
Consequents: 
 ['gender_F']
Support: 0.00199, Confidence: 1.0


Top 2 :
Antecedents: 
 ['product_name_Hermanos Elephant Garlic', 'yearly_income_$30K - $50K', 'occupation_Manual']
Consequents: 
 ['gender_M']
Support: 0.00186, Confidence: 1.0


Top 3 :
Antecedents: 
 ['product_name_Even Better Large Curd Cottage Cheese', 'customer_state_province_Yucatan']
Consequents: 
 ['gender_F']
Support: 0.00161, Confidence: 1.0


Top 4 :
Antecedents: 
 ['product_name_Plato Strawberry Jam', 'yearly_income_$70K - $90K']
Consequents: 
 ['occupation_Professional']
Support: 0.00161, Confidence: 1.0


Top 5 :
Antecedents: 
 ['product_name_Choice Tasty Candy Bar', 'yearly_income_$50K - $70K', 'gender_M']
Consequents: 
 ['occupation_Professional']
Support: 0.00161, Confidence: 1.0


Top 6 :
Antecedents: 
 ['birth_year_1950', 'product_name_Fast Low Fat Popcorn']
Consequents: 
 ['gender_F']
Support: 0.00149, Confide

**(後來沒用到)** 將該筆transaction的商品價格加總

In [31]:
# price = all_df['product_retail_price']
# print(f'Mean: {round(np.mean(price),2)}, Max: {np.max(price)}, Min: {np.min(price)}')

# lower_q = np.quantile(price,0.25,interpolation='lower') #25%
# higher_q = np.quantile(price,0.75,interpolation='higher') #75%
# print(f'Quantile: {lower_q, higher_q}')

In [32]:
# for i in range(len(all_df)):
#   price = all_df['product_retail_price'][i]

#   if price < 5.78:
#     all_df['product_retail_price'][i] = '< $5.78'
#   elif 5.78 <= price < 10.23:
#     all_df['product_retail_price'][i] = '$5.78 - $10.23'
#   elif 10.23 <= price < 13.84:
#     all_df['product_retail_price'][i] = '$10.23 - $13.84'
#   else:
#     all_df['product_retail_price'][i] = '> $13.84'

# display(all_df)

# Q4
在美國由於聖誕節，12月是購物的旺季。請探勘分析比較 12 月與 1~11月的顧客購物行為。 有哪些相似的地方，有哪些差異的地方?

In [33]:
product_partial = product_df[['product_id', 'product_name']]
transaction_partial = transaction_df[['transaction_date', 'product_id', 'customer_id', 'store_id']]

display(product_partial)
display(transaction_partial)

,product_id,product_name
0,1,Washington Berry Juice
1,2,Washington Mango Drink
2,3,Washington Strawberry Drink
3,4,Washington Cream Soda
4,5,Washington Diet Soda
...,...,...
1555,1556,CDR Creamy Peanut Butter
1556,1557,CDR Strawberry Preserves
1557,1558,CDR Extra Chunky Peanut Butter
1558,1559,CDR Apple Preserves


,transaction_date,product_id,customer_id,store_id
0,1/1/1998,4,2439,10
1,1/1/1998,11,4284,10
2,1/1/1998,12,534,10
3,1/1/1998,14,9743,10
4,1/1/1998,16,3608,10
...,...,...,...,...
182878,12/30/1998,1521,7197,11
182879,12/30/1998,1536,5223,10
182880,12/30/1998,1542,8077,10
182881,12/30/1998,1544,4485,10


In [34]:
# 12月的transaction資料
trans_12 = transaction_partial[pd.to_datetime(transaction_partial['transaction_date'], format = "%m/%d/%Y").dt.month == 12]
display(trans_12)

# 1-11月的transaction資料
trans_1 = transaction_partial[pd.to_datetime(transaction_partial['transaction_date'], format = "%m/%d/%Y").dt.month != 12]
display(trans_1)

,transaction_date,product_id,customer_id,store_id
164558,12/1/1998,6,6288,13
164559,12/1/1998,7,4629,13
164560,12/1/1998,9,1304,13
164561,12/1/1998,10,4623,13
164562,12/1/1998,11,8207,13
...,...,...,...,...
182878,12/30/1998,1521,7197,11
182879,12/30/1998,1536,5223,10
182880,12/30/1998,1542,8077,10
182881,12/30/1998,1544,4485,10


,transaction_date,product_id,customer_id,store_id
0,1/1/1998,4,2439,10
1,1/1/1998,11,4284,10
2,1/1/1998,12,534,10
3,1/1/1998,14,9743,10
4,1/1/1998,16,3608,10
...,...,...,...,...
164553,11/30/1998,1543,3201,11
164554,11/30/1998,1544,6081,10
164555,11/30/1998,1552,5658,11
164556,11/30/1998,1555,2269,11


In [35]:
trans_product_1 = pd.merge(trans_1, product_partial, on = ['product_id'])
trans_product_12 = pd.merge(trans_12, product_partial, on = ['product_id'])

trans_product_1 = trans_product_1.groupby(['transaction_date', 'customer_id', 'store_id'])['product_name'].apply(list)
display(trans_product_1)

trans_product_12 = trans_product_12.groupby(['transaction_date', 'customer_id', 'store_id'])['product_name'].apply(list)
display(trans_product_12)

transaction_date  customer_id  store_id
1/1/1998          534          10          [Jeffers Oatmeal, Robust Monthly Sports Magazi...
                  1013         10          [Club 1% Milk, Faux Products Whitening Toothpa...
                  1325         10          [Club Havarti Cheese, Club Strawberry Yogurt, ...
                  1425         10          [Club Jack Cheese, Golden Lemon Popsicles, Den...
                  1550         10          [Blue Label Noodle Soup, Bird Call 200 MG Acet...
                                                                 ...                        
9/9/1998          9638         11          [Good Chablis Wine, Consolidated Buffered Aspi...
                  9788         13          [Carrington Fajita French Fries, Tell Tale Gar...
                  9871         13          [Big Time Frozen Mushroom Pizza, Hilltop Laund...
                  10118        13          [Nationeel Avocado Dip, Moms Sliced Ham, Red W...
                  10186       

transaction_date  customer_id  store_id
12/1/1998         217          13          [Cutting Edge Sliced Turkey, Horatio Low Fat C...
                  465          13          [Blue Label Turkey Noodle Soup, Golden Popsicl...
                  484          13          [Bird Call Extra Moisture Shampoo, Tri-State G...
                  619          13          [Cutting Edge Foot-Long Hot Dogs, Horatio Choc...
                  767          4                            [Big Time Frozen Chicken Breast]
                                                                 ...                        
12/9/1998         10002        12          [Horatio Beef Jerky, Skinner Cream Soda, Plato...
                  10114        1                                 [Hilltop Laundry Detergent]
                  10161        7           [Monarch Manicotti, Tell Tale Garlic, Nationee...
                  10192        7           [Steady Conditioning Shampoo, Steady 200 MG Ib...
                  10253       

開始分析1-11月的消費行為

In [36]:
# Transform products name to one-hot encoding
TE = TransactionEncoder()
trans_product_1_te = TE.fit(trans_product_1).transform(trans_product_1)
trans_product_1_te_df = pd.DataFrame(trans_product_1_te, columns=TE.columns_)
display(trans_product_1_te_df)

,ADJ Rosy Sunglasses,Akron City Map,Akron Eyeglass Screwdriver,American Beef Bologna,American Chicken Hot Dogs,American Cole Slaw,American Corned Beef,American Foot-Long Hot Dogs,American Low Fat Bologna,American Low Fat Cole Slaw,...,Washington Apple Juice,Washington Berry Juice,Washington Cola,Washington Cranberry Juice,Washington Cream Soda,Washington Diet Cola,Washington Diet Soda,Washington Mango Drink,Washington Orange Juice,Washington Strawberry Drink
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34065,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34066,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34067,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
34068,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [73]:
# select the threshold - support & cofidence
conditions = {"Min Support": 0.0003, "Min Confidence": 1e-10}

apriori_df_1 = apriori(trans_product_1_te_df, min_support=conditions["Min Support"], use_colnames=True, low_memory=True)
apriori_df_1_top20 = apriori_df_1.sort_values(['support'], ascending=False).head(20)
apriori_df_1_top20 = apriori_df_1_top20.reset_index(drop=True)
display(apriori_df_1_top20)

association_df_1 = association_rules(apriori_df_1 , metric="confidence", min_threshold=conditions["Min Confidence"])
association_df_1 = association_df_1[['antecedents', 'consequents', 'support', 'confidence']]
association_df_1_top10 = association_df_1.sort_values(['support', 'confidence'], ascending=False).head(10)
association_df_1_top10 = association_df_1_top10.reset_index(drop=True)
display(association_df_1_top10)

,support,itemsets
0,0.004197,(Great English Muffins)
1,0.004109,(Carrington Ice Cream)
2,0.004050,(Nationeel Dried Apples)
3,0.004050,(Nationeel Fudge Brownies)
4,0.004021,(Booker String Cheese)
5,0.004021,(Ebony Mixed Nuts)
6,0.004021,(Excellent Orange Juice)
7,0.003992,(Steady Childrens Cold Remedy)
8,0.003962,(Moms Roasted Chicken)
9,0.003933,(Great Pumpernickel Bread)


,antecedents,consequents,support,confidence
0,(Carlson Low Fat Sour Cream),(Big Time Apple Cinnamon Waffles),0.000352,0.109091
1,(Big Time Apple Cinnamon Waffles),(Carlson Low Fat Sour Cream),0.000352,0.102564
2,(Tell Tale New Potatos),(Plato Strawberry Jelly),0.000323,0.098214
3,(Plato Strawberry Jelly),(Tell Tale New Potatos),0.000323,0.093220
4,(Great Wheat Bread),(Best Choice Salsa Dip),0.000323,0.088000
5,(Best Choice Salsa Dip),(Great Wheat Bread),0.000323,0.086614


開始分析12月的消費行為

In [38]:
# Transform products name to one-hot encoding
TE = TransactionEncoder()
trans_product_12_te = TE.fit(trans_product_12).transform(trans_product_12)
trans_product_12_te_df = pd.DataFrame(trans_product_12_te, columns=TE.columns_)
display(trans_product_12_te_df)

,ADJ Rosy Sunglasses,Akron City Map,Akron Eyeglass Screwdriver,American Beef Bologna,American Chicken Hot Dogs,American Cole Slaw,American Corned Beef,American Foot-Long Hot Dogs,American Low Fat Bologna,American Low Fat Cole Slaw,...,Washington Apple Juice,Washington Berry Juice,Washington Cola,Washington Cranberry Juice,Washington Cream Soda,Washington Diet Cola,Washington Diet Soda,Washington Mango Drink,Washington Orange Juice,Washington Strawberry Drink
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3776,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3777,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3778,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3779,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [69]:
# select the threshold - support & cofidence
conditions = {"Min Support": 0.0003, "Min Confidence": 1e-10}

apriori_df_12 = apriori(trans_product_12_te_df, min_support=conditions["Min Support"], use_colnames=True, low_memory=True)
apriori_df_12_top20 = apriori_df_12.sort_values(['support'], ascending=False).head(20)
apriori_df_12_top20 = apriori_df_12_top20.reset_index(drop=True)
display(apriori_df_12_top20)

association_df_12 = association_rules(apriori_df_12 , metric="confidence", min_threshold=conditions["Min Confidence"])
association_df_12 = association_df_12[['antecedents', 'consequents', 'support', 'confidence']]
association_df_12_top10 = association_df_12.sort_values(['support', 'confidence'], ascending=False).head(10)
association_df_12_top10 = association_df_12_top10.reset_index(drop=True)
display(association_df_12_top10)

,support,itemsets
0,0.006612,(Hilltop 200 MG Ibuprofen)
1,0.006083,(Booker Low Fat Cottage Cheese)
2,0.006083,(Super Grape Jam)
3,0.006083,(American Sliced Ham)
4,0.005819,(Landslide Vegetable Oil)
5,0.005819,(Moms Roasted Chicken)
6,0.005554,(Urban Large Eggs)
7,0.005554,(Tri-State Corn on the Cob)
8,0.005554,(Top Measure White Zinfandel Wine)
9,0.005554,(Sunset Paper Cups)


,antecedents,consequents,support,confidence
0,(Dollar Monthly Auto Magazine),(Fast Low Fat Cookies),0.001058,0.363636
1,(Imagine Frozen Cauliflower),(Hermanos Prepared Salad),0.001058,0.285714
2,(Fast Low Fat Cookies),(Dollar Monthly Auto Magazine),0.001058,0.210526
3,(Hermanos Prepared Salad),(Imagine Frozen Cauliflower),0.001058,0.210526
4,"(Blue Label Canned Tuna in Water, Sunset Coppe...",(American Sliced Ham),0.000793,1.000000
5,"(Blue Label Canned Tuna in Water, American Sli...",(Sunset Copper Cleaner),0.000793,1.000000
6,"(Sunset Copper Cleaner, American Sliced Ham)",(Blue Label Canned Tuna in Water),0.000793,1.000000
7,"(Imagine Frozen Cauliflower, Best Choice Low F...",(Hermanos Prepared Salad),0.000793,1.000000
8,"(Hermanos Prepared Salad, Best Choice Low Fat ...",(Imagine Frozen Cauliflower),0.000793,1.000000
9,"(Imagine Frozen Cauliflower, Hilltop Toothpaste)",(Hermanos Prepared Salad),0.000793,1.000000
